# Image Classification using Convolutional Neural Network
#### CNN
The dataset I am using is the popular dataset of images of dogs and cats. Although, this dataset is a simple dataset, the practical application of this model can be used for many real world application. For example, it can be used for identifying diseases if we have any medical images, like x-ray images, or MRI images. Other examples can be identifying counterfeits. The metrics for measurement of this model would be accuracy.

#### Importing Necessary Packages
I am using Keras Library for building this CNN model

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
from keras.models import load_model

#### Data Preprocessing
Building the Training set. The parameter shear_range, zoom_range, horizontal flip are for feature augmentation. Rescale is for feature scaling. This should convert the input images into matrix of to 1s and 0s.

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, #feature scaling of pixels
                                   shear_range=0.2, #transformation for image augmentation
                                   zoom_range=0.2, #transformation for image augmentation
                                   horizontal_flip=True)#transformation for image augmentation

train_generator = train_datagen.flow_from_directory(
    'data/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


Building the test set, in test set I am not applying the transformations to avoid overfitting. Only feature scaling


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'data/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


#### Model Building
Initializing the CNN has following steps:
1. Adding Convolution Layer
2. Pooling
3. Flattening
4. Full Connection
5. Output Layer

Note** 32 is number of feature detector, (3,3) is the size, 3 in the input shape represent colored image

In [4]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))

In [5]:
#pooling
cnn.add(tf.keras.layers.MaxPooling2D((2, 2),strides=(2, 2)))

In [6]:
#second Convolution Layer
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))

In [7]:
#Flattening
cnn.add(tf.keras.layers.Flatten())

In [8]:
#Building the fully connected model
cnn.add(tf.keras.layers.Dense(128, activation='relu'))

activation function is sigmoid as it's a binary classifier and one neuron


In [9]:
#output layer
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#### Model Fitting and Training
The metrics used is accuracy. Which is Correct Classification/ Total Classification. 

In [10]:
#compile
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
#fit
cnn.fit(x=train_generator, validation_data= test_generator,epochs=30)

Epoch 1/30
250/250 [==============================] - 67s 266ms/step - loss: 0.6665 - accuracy: 0.5914 - val_loss: 0.6239 - val_accuracy: 0.6490
Epoch 2/30
250/250 [==============================] - 27s 106ms/step - loss: 0.6133 - accuracy: 0.6633 - val_loss: 0.5874 - val_accuracy: 0.6725
Epoch 3/30
250/250 [==============================] - 25s 102ms/step - loss: 0.5630 - accuracy: 0.7153 - val_loss: 0.5397 - val_accuracy: 0.7380
Epoch 4/30
250/250 [==============================] - 25s 101ms/step - loss: 0.5356 - accuracy: 0.7320 - val_loss: 0.5167 - val_accuracy: 0.7530
Epoch 5/30
250/250 [==============================] - 26s 102ms/step - loss: 0.5044 - accuracy: 0.7517 - val_loss: 0.5150 - val_accuracy: 0.7580
Epoch 6/30
250/250 [==============================] - 26s 104ms/step - loss: 0.4862 - accuracy: 0.7669 - val_loss: 0.5040 - val_accuracy: 0.7620
Epoch 7/30
250/250 [==============================] - 27s 108ms/step - loss: 0.4714 - accuracy: 0.7745 - val_loss: 0.4825 - val_ac

The final accuracy of the model with the full dataset is 92.10%.

#### Saving the model
Saving the model. This should help us use the model locally later. As well to move it to ec2 for model deployment.

In [12]:
cnn.save('models/cnn.h5')

#### Local Function to use the model

In [13]:
#prediction
def predict_cats_dogs(image_path, model_path = 'models/cnn.h5'):
    """
    This function will predict if the image is of a dog or cat
    :param image_path: the path of the image 
    :param model_path: the path of the model
    :return: the prediction
    """
    predictor = load_model(model_path)
    test_image = utils.load_img(image_path, target_size=(64, 64))
    test_image = utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    predictions = predictor.predict(test_image)
    #print(train_generator.class_indices)
    #train_generator.class_indices
    #print(predictions)
    if predictions[0][0] == 1:
        return f"It's a Dog"
    else:
        return f"It's Cat"



#### Using the model

In [15]:
predict_cats_dogs('data/real_prediction/Rando_Cat.jpg')

1/1 [==============================] - 0s 78ms/step


"It's Cat"